In [33]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

In [2]:
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [4]:
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       270
           1       0.62      0.50      0.56        30

    accuracy                           0.92       300
   macro avg       0.79      0.73      0.76       300
weighted avg       0.91      0.92      0.92       300



C:\Users\rohit\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [5]:
report_dict = classification_report(y_test, y_pred, output_dict = True)
report_dict

{'0': {'precision': 0.9456521739130435,
  'recall': 0.9666666666666667,
  'f1-score': 0.9560439560439561,
  'support': 270.0},
 '1': {'precision': 0.625,
  'recall': 0.5,
  'f1-score': 0.5555555555555556,
  'support': 30.0},
 'accuracy': 0.92,
 'macro avg': {'precision': 0.7853260869565217,
  'recall': 0.7333333333333334,
  'f1-score': 0.7557997557997558,
  'support': 300.0},
 'weighted avg': {'precision': 0.9135869565217392,
  'recall': 0.92,
  'f1-score': 0.9159951159951161,
  'support': 300.0}}

In [6]:
import mlflow

In [7]:
report_dict['accuracy']

0.92

In [12]:
mlflow.set_experiment('First Experiment v2')
mlflow.set_tracking_uri('http://127.0.0.1:5000')

with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metrics({
        'accuracy' : report_dict['accuracy'],
        'recall_class_0' : report_dict['0']['recall'],
        'recall_class_1' : report_dict['1']['recall'],
        'f1_score_macro' : report_dict['macro avg']['f1-score']
    })

    mlflow.sklearn.log_model(lr, 'Logistic Regression')

2025/09/29 12:53:47 INFO mlflow.tracking.fluent: Experiment with name 'First Experiment v2' does not exist. Creating a new experiment.
2025/09/29 12:53:47 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/29 12:53:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run classy-dove-944 at: http://127.0.0.1:5000/#/experiments/108040543820621860/runs/ead6f8b20695475fbd712f3b007e9953
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/108040543820621860


In [13]:
log_reg = LogisticRegression(C = 1, solver = 'liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



In [14]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98       270
           1       0.95      0.70      0.81        30

    accuracy                           0.97       300
   macro avg       0.96      0.85      0.89       300
weighted avg       0.97      0.97      0.96       300



In [17]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



C:\Users\rohit\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:55:29] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [18]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

In [19]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



C:\Users\rohit\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:55:45] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [20]:
models = [
    (
        'Logistic Regression',
        {'C' : 1, 'solver' : 'liblinear'},
        LogisticRegression(C = 1, solver = 'liblinear'),
        (X_train, y_train),
        (X_test, y_test)
    ),

    (
        'Random Forest',
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(n_estimators=30, max_depth=3), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train, y_train),
        (X_test, y_test)
    ),
(
    "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [21]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict = True)
    reports.append(report)

C:\Users\rohit\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:56:15] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\rohit\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:56:15] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [22]:
reports

[{'0': {'precision': 0.9454545454545454,
   'recall': 0.9629629629629629,
   'f1-score': 0.9541284403669725,
   'support': 270.0},
  '1': {'precision': 0.6,
   'recall': 0.5,
   'f1-score': 0.5454545454545454,
   'support': 30.0},
  'accuracy': 0.9166666666666666,
  'macro avg': {'precision': 0.7727272727272727,
   'recall': 0.7314814814814814,
   'f1-score': 0.749791492910759,
   'support': 300.0},
  'weighted avg': {'precision': 0.9109090909090909,
   'recall': 0.9166666666666666,
   'f1-score': 0.91326105087573,
   'support': 300.0}},
 {'0': {'precision': 0.96415770609319,
   'recall': 0.9962962962962963,
   'f1-score': 0.9799635701275046,
   'support': 270.0},
  '1': {'precision': 0.9523809523809523,
   'recall': 0.6666666666666666,
   'f1-score': 0.7843137254901961,
   'support': 30.0},
  'accuracy': 0.9633333333333334,
  'macro avg': {'precision': 0.9582693292370712,
   'recall': 0.8314814814814815,
   'f1-score': 0.8821386478088503,
   'support': 300.0},
  'weighted avg': {'prec

In [34]:
mlflow.set_tracking_uri('http://127.0.0.1:5000/')
mlflow.set_experiment('Anomaly Detection2')

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]

    with mlflow.start_run(run_name = model_name):
        mlflow.log_params(params)
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_class_1', report['1']['recall'])
        mlflow.log_metric('recall_class_0', report['0']['recall'])
        mlflow.log_metric('f1_score_macro', report['macro avg']['f1-score'])

        if 'XGB' in model_name:
            mlflow.xgboost.log_model(model, 'model')
        else:
            mlflow.sklearn.log_model(model, 'model')


2025/09/29 13:34:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/29 13:34:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/407083464553164734/runs/0971d83970124300909ca318130d9d3f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/407083464553164734


2025/09/29 13:34:50 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/29 13:34:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/407083464553164734/runs/b399e83d57e640a7b3191dc231ddf051
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/407083464553164734


2025/09/29 13:34:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
C:\Users\rohit\anaconda3\Lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [13:34:58] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/09/29 13:35:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/407083464553164734/runs/69d7ee4c2d384cb7a7a870459da6aa03
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/407083464553164734


2025/09/29 13:35:07 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
C:\Users\rohit\anaconda3\Lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [13:35:07] WARNING: C:\b\abs_d97hy_84m6\croot\xgboost-split_1749630932152\work\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/09/29 13:35:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/407083464553164734/runs/0f2925470bf34e0898a74884fb79de81
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/407083464553164734


In [36]:
model_name = 'XGB-Smote'
run_id = input('Enter Run ID')
model_uri = f"runs:/{run_id}/model"
result = mlflow.register_model(
    model_uri, model_name
)

Enter Run ID 0f2925470bf34e0898a74884fb79de81


Registered model 'XGB-Smote' already exists. Creating a new version of this model...
2025/09/29 13:36:48 WARNING mlflow.tracking._model_registry.fluent: Run with id 0f2925470bf34e0898a74884fb79de81 has no artifacts at artifact path 'model', registering model based on models:/m-b849321f7a534df4aea404ff66d0b62b instead
2025/09/29 13:36:48 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1
Created version '1' of model 'XGB-Smote'.


In [40]:
model_version = 1
model_uri = f'models:/{model_name}/{model_version}'

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

In [41]:
model_version = 1
model_uri = f'models:/{model_name}@challenger'

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

In [42]:
dev_model_uri = f'models:/{model_name}/{model_version}'
prod_model = 'anomaly-detection-prod'

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri = dev_model_uri , dst_name = prod_model)

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1759181061829, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1759181061829, metrics=None, model_id=None, name='anomaly-detection-prod', params=None, run_id='0f2925470bf34e0898a74884fb79de81', run_link='', source='models:/XGB-Smote/1', status='READY', status_message=None, tags={}, user_id='', version='1'>